In [38]:
import pandas
import ast

In [13]:

def insert_separator(text, target_word, separator, capture_only_seperate_word=True):
    result = []
    if capture_only_seperate_word:
        ## This case capture only "... target_word ..." not "... [other characters]target_word[other characters] ..."
        words = text.split()
        for word in words:
            if word == target_word:
                result.append(f"\n{separator}{word}\n")
            else:
                result.append(word)
        return ' '.join(result)   

    else:     
        index = 0
        while index < len(text):
            if text[index:].startswith(target_word):
                result.append(f"\n{separator}{target_word}")
                index += len(target_word)
                # 조사나 다른 문자가 붙어있는지 확인
                while index < len(text) and not text[index].isspace():
                    result.append(text[index])
                    index += 1
                result.append("\n")
            else:
                result.append(text[index])
                index += 1
        return ''.join(result)





In [14]:

original_text = "안녕하세요 파이썬 프로그래밍은 재미있습니다 파이썬은 강력합니다"
target = "파이썬"
separator = ">>>"

modified_text = insert_separator(original_text, target, separator)
print(modified_text)

안녕하세요 
>>>파이썬
 프로그래밍은 재미있습니다 파이썬은 강력합니다


In [19]:
import pandas as pd
import re

def extract_types(context):
    # 정규 표현식을 사용하여 'type' 값을 추출
    types = re.findall(r"'type'\s*:\s*'(\w+)'", context)
    return types



In [20]:

# 예시 DataFrame 생성
df = pd.DataFrame({
    'context': [
        "[{'type':'aaa', 'text':'Content: Question: kllkllbbaa kflalf  Response: ffaff \n Content:  Question: fffa Response: ffkkkbb'},  {'type':'bbb', 'text':'Content: Question: kllkllbbaa kflalf  Response: ffaff \n Content:  Question: fffa Response: ffkkkbb'}, ,  {'type':'ccc', 'text':'Page Content: fffkllbbaa kflalf  Response: ffaff \n Page Content:  fddfkkkbb'}]"
    ]
})

# 'context' 열에서 type 리스트 추출
df['extracted_types'] = df['context'].apply(extract_types)

print(df['extracted_types'])

0    [aaa, bbb, ccc]
Name: extracted_types, dtype: object


In [21]:
import pandas as pd
import re

def count_keywords(text, keywords):
    counts = {keyword: 0 for keyword in keywords}
    for keyword in keywords:
        counts[keyword] = len(re.findall(re.escape(keyword), text))
    return counts



In [22]:
# 키워드 목록 설정 (이 부분은 필요에 따라 수정 가능합니다)
keywords = ['Content', 'Page Content']

# 예시 DataFrame 생성
df = pd.DataFrame({
    'context': [
        "[{'type':'aaa', 'text':'Content: Question: kllkllbbaa kflalf  Response: ffaff \n Content:  Question: fffa Response: ffkkkbb'},  {'type':'bbb', 'text':'Content: Question: kllkllbbaa kflalf  Response: ffaff \n Content:  Question: fffa Response: ffkkkbb'}, ,  {'type':'ccc', 'text':'Page Content: fffkllbbaa kflalf  Response: ffaff \n Page Content:  fddfkkkbb'}]",
        "[{'type':'ddd', 'text':'Page Content: abcdef \n Content: ghijkl'}]"
    ]
})

# 각 행의 'context' 열에서 키워드 카운트
df['keyword_counts'] = df['context'].apply(lambda x: count_keywords(x, keywords))

# 결과 출력
print(df[['context', 'keyword_counts']])

# 키워드별로 별도의 열 생성 (선택사항)
for keyword in keywords:
    df[f'{keyword}_count'] = df['keyword_counts'].apply(lambda x: x[keyword])

print(df)

                                             context  \
0  [{'type':'aaa', 'text':'Content: Question: kll...   
1  [{'type':'ddd', 'text':'Page Content: abcdef \...   

                      keyword_counts  
0  {'Content': 6, 'Page Content': 2}  
1  {'Content': 2, 'Page Content': 1}  
                                             context  \
0  [{'type':'aaa', 'text':'Content: Question: kll...   
1  [{'type':'ddd', 'text':'Page Content: abcdef \...   

                      keyword_counts  Content_count  Page Content_count  
0  {'Content': 6, 'Page Content': 2}              6                   2  
1  {'Content': 2, 'Page Content': 1}              2                   1  


In [ ]:
10

In [23]:
df

,context,keyword_counts,Content_count,Page Content_count
0,"[{'type':'aaa', 'text':'Content: Question: kll...","{'Content': 6, 'Page Content': 2}",6,2
1,"[{'type':'ddd', 'text':'Page Content: abcdef \...","{'Content': 2, 'Page Content': 1}",2,1


In [85]:
def split_str_with_start_delimiter(target_str, delimiter_str):
    output_list_str = []
    
    # 문자열의 시작부터 검색 시작
    start = 0
    while True:
        # 구분자의 위치 찾기
        index = target_str.find(delimiter_str, start)
        
        if index == -1:  # 구분자를 더 이상 찾을 수 없으면
            # 남은 문자열을 리스트에 추가하고 종료
            if start < len(target_str):
                output_list_str.append(target_str[start:])
            break
        
        # 구분자 이전의 문자열을 리스트에 추가 (비어있지 않은 경우)
        if start < index:
            output_list_str.append(target_str[start:index])
        
        # 다음 검색 시작 위치를 구분자 다음으로 이동
        start = index + len(delimiter_str)
    
    return output_list_str

def extract_list_context_from_target_str(target_str, delimiter_str):
    ## example input: text:bbbbb...
    list_context = split_str_with_start_delimiter(target_str, delimiter_str)[1:]
    list_context = [context.strip() for context in list_context]

    return list_context


def extract_wholelist_context_from_contexts(contexts_str, context_type_key = 'type', context_text_key='text'):
    ## example input (contexts_str): "[{type:aaa, text:bbbbb...}, {type:aaa, text:bbbbb...}]""
    list_dict_context = ast.literal_eval(contexts_str)

    list_whole_context = []

    print("list_dict_context:", list_dict_context)

    for dict_context in list_dict_context:
        print("dict_context:", dict_context)
        context_type = dict_context[context_type_key]
        context_text = dict_context[context_text_key]
        print(f"context_type: {context_type}")
        print(f"context_text: {context_text}")
        if context_type == 'aaa':
            delimiter_str = 'Content:'
        elif context_type == 'ccc':
            delimiter_str = 'Page Content:'
        else:
            delimiter_str = 'Content:'
        list_context = extract_list_context_from_target_str(context_text, delimiter_str)
        # print("list_context:", list_context)
        list_whole_context.extend(list_context)

    return list_whole_context


In [86]:
input_str = "[{'type':'aaa', 'text':'Content: Question: kllkllbbaa kflalf \\n Response: ffaff Content:  Question: fffa Response: ffkkkbb'}]"

ast.literal_eval(input_str)

[{'type': 'aaa',
  'text': 'Content: Question: kllkllbbaa kflalf \n Response: ffaff Content:  Question: fffa Response: ffkkkbb'}]

In [87]:
input_str = "[{'type':'aaa', 'text':'Content: Question: kllkllbbaa kflalf  Response: ffaff \\n Content:  Question: fffa Response: ffkkkbb'},  {'type':'bbb', 'text':'Content: Question: kllkllbbaa kflalf  Response: ffaff \\n Content:  Question: fffa Response: ffkkkbb'}, {'type':'ccc', 'text':'Page Content: fffkllbbaa kflalf  Response: ffaff \\n Page Content:  fddfkkkbb'}]"

# input_str = "[{'type':'aaa', 'text':'Content: Question: kllkllbbaa kflalf  Response: ffaff Content:  Question: fffa Response: ffkkkbb'}]"

# ast.literal_eval(input_str)

# list_context = extract_list_context(input_str, 'Content:')

list_context = extract_wholelist_context_from_contexts(input_str, context_type_key = 'type', context_text_key='text')

list_dict_context: [{'type': 'aaa', 'text': 'Content: Question: kllkllbbaa kflalf  Response: ffaff \n Content:  Question: fffa Response: ffkkkbb'}, {'type': 'bbb', 'text': 'Content: Question: kllkllbbaa kflalf  Response: ffaff \n Content:  Question: fffa Response: ffkkkbb'}, {'type': 'ccc', 'text': 'Page Content: fffkllbbaa kflalf  Response: ffaff \n Page Content:  fddfkkkbb'}]
dict_context: {'type': 'aaa', 'text': 'Content: Question: kllkllbbaa kflalf  Response: ffaff \n Content:  Question: fffa Response: ffkkkbb'}
context_type: aaa
context_text: Content: Question: kllkllbbaa kflalf  Response: ffaff 
 Content:  Question: fffa Response: ffkkkbb
dict_context: {'type': 'bbb', 'text': 'Content: Question: kllkllbbaa kflalf  Response: ffaff \n Content:  Question: fffa Response: ffkkkbb'}
context_type: bbb
context_text: Content: Question: kllkllbbaa kflalf  Response: ffaff 
 Content:  Question: fffa Response: ffkkkbb
dict_context: {'type': 'ccc', 'text': 'Page Content: fffkllbbaa kflalf  Res

In [88]:
list_context

['Question: fffa Response: ffkkkbb',
 'Question: fffa Response: ffkkkbb',
 'fddfkkkbb']

In [47]:
import ast

def convert_to_triple_quotes_ast(input_str):
    try:
        # 문자열 내용 평가
        content = ast.literal_eval(input_str)
        # 삼중 따옴표로 감싸기
        return f'''{content}'''
    except (SyntaxError, ValueError):
        # 평가 실패 시 원래 문자열 반환
        return input_str

# 예시 사용
original_str = '"This is a string with "escaped" quotes"'
converted_str = convert_to_triple_quotes_ast(original_str)
print(converted_str)

"This is a string with "escaped" quotes"


In [48]:
convert_to_triple_quotes_ast(input_str)

"[{'type':'aaa', 'text':'Content: Question: kllkllbbaa kflalf  Response: ffaff \n Content:  Question: fffa Response: ffkkkbb'},  {'type':'bbb', 'text':'Content: Question: kllkllbbaa kflalf  Response: ffaff \n Content:  Question: fffa Response: ffkkkbb'}, {'type':'ccc', 'text':'Page Content: fffkllbbaa kflalf  Response: ffaff \n Page Content:  fddfkkkbb'}]"

In [43]:

list_context = extract_wholelist_context_from_contexts(convert_to_triple_quotes_ast(input_str), context_type_key = 'type', context_text_key='text')

SyntaxError: unterminated string literal (detected at line 1) (<unknown>, line 1)

In [37]:
list_context

['Question: kllkllbbaa kflalf  Response: ffaff',
 "Question: fffa Response: ffkkkbb'},  {'type':'bbb', 'text':'",
 'Question: kllkllbbaa kflalf  Response: ffaff',
 "Question: fffa Response: ffkkkbb'}, ,  {'type':'ccc', 'text':'Page",
 'fffkllbbaa kflalf  Response: ffaff \n Page',
 "fddfkkkbb'}]"]

In [30]:
list_context


[' Question: kllkllbbaa kflalf  Response: ffaff \n ',
 "  Question: fffa Response: ffkkkbb'},  {'type':'bbb', 'text':'",
 ' Question: kllkllbbaa kflalf  Response: ffaff \n ',
 "  Question: fffa Response: ffkkkbb'}, ,  {'type':'ccc', 'text':'Page ",
 ' fffkllbbaa kflalf  Response: ffaff \n Page ',
 "  fddfkkkbb'}]"]

In [ ]:
def extract_list_context(contexts):
    
